# Replace LEC and MWTP's CI with Statistical

In [1]:
%pwd

'/home/pj24001881/share/DP15_Article/12_PyCode'

In [2]:
%cd ..

/home/pj24001881/share/DP15_Article


/home/pj24001881/ku40001335/miniconda3/envs/tf/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [3]:
import numpy as np
import pandas as pd

## load data

In [24]:
result_df_ndvi = pd.read_excel('13_Results/PdpConnectionCoef/PdpConnection_NDVI_v2.xlsx', index_col = 0)

In [25]:
result_df_ndvi['significant'] = (result_df_ndvi['coeff'].abs() - result_df_ndvi['SE_coeff'] * 1.96) >= 0

In [26]:
result_df_ndvi_insignificant = result_df_ndvi[result_df_ndvi['significant'] == False]

In [27]:
result_df_ndvi_insignificant[result_df_ndvi_insignificant['r2'] > 0.5]

,lat,lon,coeff,intercept,r2,SE_coeff,significant


In [28]:
result_df_ndvi_insignificant.shape

(36673, 7)

In [29]:
result_df_ndvi.head()

,lat,lon,coeff,intercept,r2,SE_coeff,significant
0,35.918037,140.013626,0.000442,0.000004,0.750660,0.000041,True
1,35.604172,139.486008,-0.001507,0.000117,0.974180,0.000039,True
2,35.170216,138.682770,0.001823,-0.002013,0.427581,0.000338,True
3,34.741104,137.707062,0.002899,-0.000212,0.994833,0.000033,True
4,33.515968,130.407059,0.001064,0.000420,0.904825,0.000055,True


In [30]:
result_df_ndvi.loc[result_df_ndvi['r2'] < 0.5, 'coeff'] = 0

In [31]:
result_df_ndvi_loc = result_df_ndvi.drop_duplicates()

In [32]:
result_df_ndvi_loc = result_df_ndvi_loc[['lat', 'lon', 'coeff', 'SE_coeff']]

In [33]:
result_df_ndvi_loc.columns = ['lat', 'lon', 'coeff_ndvi', 'SE_coeff_ndvi']

In [44]:
result_df_ntl = pd.read_excel('13_Results/PdpConnectionCoef/PdpConnection_NTL_v2.xlsx', index_col = 0)

In [45]:
result_df_ntl['significant'] = (result_df_ntl['coeff'].abs() - result_df_ntl['SE_coeff'] * 1.96) >= 0

In [46]:
result_df_ntl_insignificant = result_df_ntl[result_df_ntl['significant'] == False]

In [47]:
result_df_ntl_insignificant[result_df_ntl_insignificant['r2'] > 0.5]

,lat,lon,coeff,intercept,r2,SE_coeff,significant


In [48]:
result_df_ntl_insignificant.shape

(15279, 7)

In [49]:
result_df_ntl.loc[result_df_ntl['r2'] < 0.5, 'coeff'] = 0

In [50]:
result_df_ntl_loc = result_df_ntl.drop_duplicates()

In [51]:
result_df_ntl_loc = result_df_ntl_loc[['lat', 'lon', 'coeff', 'SE_coeff']]

In [52]:
result_df_ntl_loc.columns = ['lat', 'lon', 'coeff_ntl', 'SE_coeff_ntl']

In [54]:
result_df_income = pd.read_excel('13_Results/PdpConnectionCoef/PdpConnection_income_indiv_v2.xlsx', index_col = 0)

In [55]:
result_df_income['significant'] = (result_df_income['coeff'].abs() - result_df_income['SE_coeff'] * 1.96) >= 0

In [56]:
result_df_income_insignificant = result_df_income[result_df_income['significant'] == False]

In [57]:
result_df_income_insignificant[result_df_income_insignificant['r2'] > 0.5]

,lat,lon,coeff,intercept,r2,SE_coeff,significant


In [58]:
result_df_income_insignificant.shape

(1509, 7)

In [59]:
result_df_income.loc[result_df_income['r2'] < 0.5, 'coeff'] = 0

In [60]:
result_df_income_loc = result_df_income.drop_duplicates()

In [61]:
result_df_income_loc = result_df_income_loc[['lat', 'lon', 'coeff', 'SE_coeff']]

In [62]:
result_df_income_loc.columns = ['lat', 'lon', 'coeff_income', 'SE_coeff_income']

In [63]:
merged_df = pd.merge(result_df_ndvi_loc, result_df_ntl_loc, on=['lat', 'lon'], how='left')
result_df_loc_merge = pd.merge(merged_df, result_df_income_loc, on=['lat', 'lon'], how='left')

In [66]:
result_df_loc_merge.columns

Index(['lat', 'lon', 'coeff_ndvi', 'SE_coeff_ndvi', 'coeff_ntl',
       'SE_coeff_ntl', 'coeff_income', 'SE_coeff_income', 'MWPT_NDVI',
       'MWPT_NTL'],
      dtype='object')

In [87]:
result_df_loc_merge['MWPT_NDVI'] = result_df_loc_merge['coeff_ndvi'] / result_df_loc_merge['coeff_income'] / 120.96 * 1_000_000

In [88]:
result_df_loc_merge['MWPT_NTL'] = result_df_loc_merge['coeff_ntl'] / result_df_loc_merge['coeff_income'] / 120.96 * 1_000_000

In [89]:
a = result_df_loc_merge['coeff_ndvi']
SE_a = result_df_loc_merge['SE_coeff_ndvi']
b = result_df_loc_merge['coeff_income']
SE_b = result_df_loc_merge['SE_coeff_income']

SE_ratio = np.sqrt((SE_a / b) ** 2 + ((a * SE_b) / (b ** 2)) ** 2)
result_df_loc_merge['SE_MWPT_NDVI'] = SE_ratio / 120.96 * 1_000_000

In [90]:
a = result_df_loc_merge['coeff_ntl']
SE_a = result_df_loc_merge['SE_coeff_ntl']
b = result_df_loc_merge['coeff_income']
SE_b = result_df_loc_merge['SE_coeff_income']

SE_ratio = np.sqrt((SE_a / b) ** 2 + ((a * SE_b) / (b ** 2)) ** 2)
result_df_loc_merge['SE_MWPT_NTL'] = SE_ratio / 120.96 * 1_000_000

In [91]:
result_df_loc_merge.head()

,lat,lon,coeff_ndvi,SE_coeff_ndvi,coeff_ntl,SE_coeff_ntl,coeff_income,SE_coeff_income,MWPT_NDVI,MWPT_NTL,SE_MWPT_NDVI,SE_MWPT_NTL
0,24.301767,123.869034,0.000623,0.000022,0.002606,0.000275,0.012935,0.000747,397.937368,1665.706083,26.918828,200.370832
1,24.332844,124.162926,0.000623,0.000022,0.002606,0.000275,0.012935,0.000747,397.937368,1665.706083,26.918828,200.370832
2,24.333654,123.762947,0.000623,0.000022,0.002606,0.000275,0.012935,0.000747,397.937368,1665.706083,26.918828,200.370832
3,24.337803,124.143456,0.000623,0.000022,0.002606,0.000275,0.012935,0.000747,397.937368,1665.706083,26.918828,200.370832
4,24.338045,124.155281,0.000623,0.000022,0.002606,0.000275,0.012935,0.000747,397.937368,1665.706083,26.918828,200.370832


In [92]:
mask = result_df_loc_merge['MWPT_NDVI'].abs() - result_df_loc_merge['SE_MWPT_NDVI'] * 1.96 < 0
result_df_loc_merge.loc[mask, 'MWPT_NDVI'] = 0

In [93]:
mask = result_df_loc_merge['MWPT_NTL'].abs() - result_df_loc_merge['SE_MWPT_NTL'] * 1.96 < 0
result_df_loc_merge.loc[mask, 'MWPT_NTL'] = 0

In [94]:
result_df_loc_merge = result_df_loc_merge.groupby(['lat', 'lon']).mean()

In [95]:
result_df_loc_merge = result_df_loc_merge.reset_index()

In [96]:
result_df_loc_merge = result_df_loc_merge.replace([np.inf, -np.inf], 0)
result_df_loc_merge = result_df_loc_merge.fillna(0)

In [97]:
result_df_loc_merge.shape

(50852, 12)

In [98]:
result_df_loc_merge.describe()

,lat,lon,coeff_ndvi,SE_coeff_ndvi,coeff_ntl,SE_coeff_ntl,coeff_income,SE_coeff_income,MWPT_NDVI,MWPT_NTL,SE_MWPT_NDVI,SE_MWPT_NTL
count,50852.000000,50852.000000,50852.000000,50852.000000,50852.000000,50852.000000,50852.000000,50852.000000,5.085200e+04,5.085200e+04,5.085200e+04,5.085200e+04
mean,35.773296,136.851359,0.000364,0.000122,-0.002037,0.000274,0.014759,0.001097,3.745309e+02,-1.439620e+03,3.375118e+02,6.804090e+02
std,2.480900,3.331468,0.002655,0.000147,0.006116,0.000512,0.011206,0.000859,7.490753e+04,8.195933e+04,1.149361e+04,1.366199e+04
min,24.301767,123.762947,-0.072164,0.000000,-0.125678,0.000010,-0.060621,0.000000,-8.448932e+06,-1.175134e+07,0.000000e+00,0.000000e+00
25%,34.667910,135.207962,0.000000,0.000052,-0.002577,0.000060,0.008527,0.000635,0.000000e+00,-1.523141e+03,3.522346e+01,5.609525e+01
50%,35.226959,136.923462,0.000000,0.000084,-0.001347,0.000111,0.013553,0.000906,0.000000e+00,-7.235523e+02,6.350053e+01,1.022510e+02
75%,36.277805,139.635010,0.001134,0.000143,0.000000,0.000259,0.018700,0.001279,6.117558e+02,0.000000e+00,1.332613e+02,2.368855e+02
max,45.511021,145.742340,0.054274,0.010829,0.093483,0.013091,0.162315,0.015791,1.183274e+07,4.179183e+06,2.023378e+06,1.989509e+06


In [104]:
lines = []
for name in ['coeff_ndvi', 'coeff_ntl', 'coeff_income', 'MWPT_NDVI', 'MWPT_NTL']:
    lower = result_df_loc_merge[name].quantile(0.005)
    upper = result_df_loc_merge[name].quantile(0.995)
    trimmed_mean = result_df_loc_merge[(result_df_loc_merge[name] >= lower) & (result_df_loc_merge[name] <= upper)][name].mean()
    trimmed_std = result_df_loc_merge[(result_df_loc_merge[name] >= lower) & (result_df_loc_merge[name] <= upper)][name].std()
    
    line = [np.mean(result_df_loc_merge[name]), trimmed_mean, trimmed_std, np.quantile(result_df_loc_merge[name], .005),
            np.quantile(result_df_loc_merge[name], .025), np.quantile(result_df_loc_merge[name], .5),
            np.quantile(result_df_loc_merge[name], .25), np.quantile(result_df_loc_merge[name], .50), 
            np.quantile(result_df_loc_merge[name], .75), np.quantile(result_df_loc_merge[name], .95),
            np.quantile(result_df_loc_merge[name], .975), np.quantile(result_df_loc_merge[name], .995),]
    lines.append(line)

In [105]:
data_summary = pd.DataFrame(lines)

In [110]:
data_summary.columns = ['Mean', "Trimmed Mean", "Trimmed Std", "0.5% Quantile", "2.5% Quantile", "5% Quantile",
                        "25% Quantile", "50% Quantile",
                        "75% Quantile", "95% Quantile", "97.5% Quantile", "99.5% Quantile"]

In [111]:
data_summary.index = ['Local Marginal Effect Coefficient of NDVI', 'Local Marginal Effect Coefficient NTL', 'Local Marginal Effect  Coefficient Income', 'MWTP of NDVI', 'MWTP of NTL']

In [112]:
data_summary.T

,Local Marginal Effect Coefficient of NDVI,Local Marginal Effect Coefficient NTL,Local Marginal Effect Coefficient Income,MWTP of NDVI,MWTP of NTL
Mean,0.000364,-0.002037,0.014759,374.530919,-1439.619840
Trimmed Mean,0.000364,-0.001953,0.014546,203.562899,-1090.419470
Trimmed Std,0.001954,0.004242,0.009918,2074.334630,4130.100966
0.5% Quantile,-0.009340,-0.034131,-0.006948,-20813.745572,-42463.994702
2.5% Quantile,-0.004348,-0.014922,-0.001145,-4151.674070,-10632.245165
5% Quantile,0.000000,-0.001347,0.013553,0.000000,-723.552285
25% Quantile,0.000000,-0.002577,0.008527,0.000000,-1523.141016
50% Quantile,0.000000,-0.001347,0.013553,0.000000,-723.552285
75% Quantile,0.001134,0.000000,0.018700,611.755805,0.000000
95% Quantile,0.003586,0.002205,0.033662,2645.895719,1577.095847


In [102]:
data_summary.T.to_excel('13_Results/Table02_summaryTable.xlsx')